Este archivo no esta del todo completo y no debe ser corrido, el viejo archivo que creo al original annotations.xlsx ya no existe y no lo pude encontrar en el repo

In [1]:
import librosa
import os
import pandas as pd
import numpy as np

# Set current working directory to parent folder
os.chdir(os.path.abspath(".."))

In [2]:
# Lee el archivo que identifica como los audios estan divididos
df_conv = pd.read_csv(f"data/MSPCORPUS/Time_Labels/conversation_parts.txt", delimiter=";", header=None, names=['Conversation_Part', 'start_time', 'end_time'])

# Creamos dos columnas con el podcast number y la parte de ese podcast
df_conv['PC_Num'] = df_conv['Conversation_Part'].apply(lambda x: x[17:21]).astype(int)
df_conv['Part_Num'] = df_conv['Conversation_Part'].apply(lambda x: x[22:23]).astype(int)

mem = {}

def add_sync_time_columns(row):
    if row.Part_Num == 1:
        st = 0
        mem[row.PC_Num] = row.start_time
    else:
        st = row.start_time - mem[row.PC_Num]
        
    row['m_start_time'] = st
    row['m_end_time'] = row.end_time - mem[row.PC_Num]
    
    return row

# Una fila con los audios en formato inicio: 0 y final: final - inicio
df_conv = df_conv.apply(lambda row: add_sync_time_columns(row), axis=1)
df_conv = df_conv[['Conversation_Part','start_time', 'end_time', 'm_start_time', 'm_end_time', 'PC_Num', 'Part_Num']]
df_conv['Audio_Name'] = df_conv['Conversation_Part'].apply(lambda x: x[0:21]) + ".wav"

In [3]:
df_conv.drop

,Conversation_Part,start_time,end_time,m_start_time,m_end_time,PC_Num,Part_Num,Audio_Name
0,MSP-Conversation_0021_1,10.0000,316.0304,0.0000,306.0304,21,1,MSP-Conversation_0021.wav
1,MSP-Conversation_0021_2,316.0304,612.2578,306.0304,602.2578,21,2,MSP-Conversation_0021.wav
2,MSP-Conversation_0021_3,612.2578,915.2362,602.2578,905.2362,21,3,MSP-Conversation_0021.wav
3,MSP-Conversation_0021_4,915.2362,1148.0000,905.2362,1138.0000,21,4,MSP-Conversation_0021.wav
4,MSP-Conversation_0023_1,2998.0000,3341.9248,0.0000,343.9248,23,1,MSP-Conversation_0023.wav
...,...,...,...,...,...,...,...,...
690,MSP-Conversation_1523_4,3373.3633,3608.8569,845.8913,1081.3849,1523,4,MSP-Conversation_1523.wav
691,MSP-Conversation_1560_1,574.4970,850.6519,0.0000,276.1549,1560,1,MSP-Conversation_1560.wav
692,MSP-Conversation_1560_2,850.6519,1229.0000,276.1549,654.5030,1560,2,MSP-Conversation_1560.wav
693,MSP-Conversation_1568_1,0.0000,390.1417,0.0000,390.1417,1568,1,MSP-Conversation_1568.wav


In [3]:
df_annotations = pd.read_excel(f'data/annotations.xlsx')

In [4]:
df_annotations_2 = pd.merge(left = df_annotations[['Name','Emotion','Annotator','PC_Num','Part_Num']], 
                                 right = df_conv[['start_time','end_time','m_start_time','m_end_time','Audio_Name','PC_Num','Part_Num']],
                                 how = 'left', 
                                 on = ['PC_Num','Part_Num'])

In [5]:
df_annotations_2['start_time'] = df_annotations_2['m_start_time']
df_annotations_2['end_time'] = df_annotations_2['m_end_time']

df_annotations_2.drop(['m_start_time','m_end_time'], axis = 1, inplace = True)

In [6]:
df_annotations_2.to_excel('annotations_2.xlsx', index = False)

Time Fix

In [9]:
# Carga del archivo directorio con datos necesarios
df_annotations = pd.read_excel("data/annotations.xlsx")

In [2]:
with open('data/MSPCORPUS/Time_Labels/conversation_parts.txt') as f:
    data_file = f.readlines()

In [10]:
df_data_file = pd.DataFrame(data_file,columns = ['Name'])

In [27]:
df_data_file['start'] = df_data_file['Name'].str.split(';').str[1]
df_data_file['end'] = df_data_file['Name'].str.split(';').str[2]
df_data_file['Name'] = df_data_file['Name'].str.split(';').str[0]
df_data_file['end'] = df_data_file['end'].str.replace('\n','')

In [43]:
df_annotations = df_annotations.drop(['start_time','end_time'], axis = 1)

In [45]:
df_annotations = pd.merge(df_annotations,df_data_file, how = 'left' , left_on = df_annotations['Name'].str[:23], right_on = 'Name')

In [47]:
df_annotations = df_annotations.drop(['Name','Name_y'], axis = 1)

In [50]:
df_annotations = df_annotations.rename({'Name_x':'Name'}, axis = 1)

In [53]:
df_annotations = df_annotations.rename({'start':'start_time','end':'end_time'}, axis = 1)

In [54]:
# No correr
#df_annotations.to_excel('data/annotations.xlsx')